In [40]:
import pandas as pd
train = pd.read_csv('train.csv')
val = pd.read_csv('val.csv')
train_oversampling = pd.read_csv('train_oversampling.csv')
test = pd.read_csv('test.csv')
y_train= train['classe']
y_train= train_oversampling['classe']
y_val=val['classe']
y_test=test['classe']

print(f"Test set: {len(test)} exemplos, classes: {test['classe'].value_counts().to_dict()}")
#train_oversampling = pd.read_csv('train_oversampling.csv') acho que nao é preciso 

Test set: 1858 exemplos, classes: {0: 1273, -1: 417, 1: 168}


# Identificar exemplos que representam bem cada classe

## abordagem através da entropia da classificação de cada exemplo

classificar os exemplos de treino com todos os modelos de machine

In [19]:
import optuna
optuna.logging.set_verbosity(optuna.logging.CRITICAL)

from sklearn.feature_extraction.text import CountVectorizer
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

def objective_count_vec(trial, conj_train, conj_val):
    # Parâmetros a otimizar
    max_features = trial.suggest_int("max_features", 500, 5000)
    ngram_min = trial.suggest_int("ngram_min", 1, 2)
    ngram_max = trial.suggest_int("ngram_max", ngram_min, 3)  # garante ngram_max >= ngram_min

    # CountVectorizer com parâmetros sugeridos
    c_vect = CountVectorizer(
        token_pattern=r"(?u)\b\w+\b|[^\w\s]",
        ngram_range=(ngram_min, ngram_max),
        strip_accents='unicode',
        max_features=max_features
    )

    # Transformar textos
    text_train = conj_train['sentences'].apply(lambda tokens: ' '.join(tokens))
    text_val   = conj_val['sentences'].apply(lambda tokens: ' '.join(tokens))

    X_train = c_vect.fit_transform(text_train)
    X_val   = c_vect.transform(text_val)

    y_train = conj_train['classe'] 
    y_val   = conj_val['classe']

    # Treinar Logistic Regression
    model = LogisticRegression(max_iter=500)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    # Métrica: F1 macro
    score = f1_score(y_val, preds, average='macro')
    return score 

def count_vec(train, val, test):

    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_count_vec(trial, train,val),
                   n_trials=30, n_jobs=-1, show_progress_bar=False)
    
    best_params=study.best_params
    print("Melhores parâmetros:", study.best_params)
    print("Melhor F1:", study.best_value)
    
    c_vect = CountVectorizer(
        token_pattern=r"(?u)\b\w+\b|[^\w\s]",
        ngram_range=(best_params['ngram_min'], best_params['ngram_max']),
        strip_accents='unicode',
        max_features=best_params['max_features']
    )

    # Transformar textos
    text_train = train['sentences'].apply(lambda tokens: ' '.join(tokens))
    text_val = val['sentences'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['sentences'].apply(lambda tokens: ' '.join(tokens))

    # Fit no treino, transform nos outros conjuntos
    X_train = c_vect.fit_transform(text_train)
    X_val = c_vect.transform(text_val)
    X_test = c_vect.transform(text_test)

    return X_train, X_val, X_test

X_train_count_vec, X_val_count_vec, _ = count_vec(train, val, test)

c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also re

Melhores parâmetros: {'max_features': 2611, 'ngram_min': 1, 'ngram_max': 2}
Melhor F1: 0.6037149861818661


In [35]:
#aplicar modelos de ML
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

#NAIVE BAYES ------------------------------
def objective_naive_MN(trial, X_train, X_val, y_train, y_val):
    # Hiperparâmetros a otimizar
    alpha = trial.suggest_float("alpha", 1e-3, 2.0, log=True)
    
    # Modelo
    clf = MultinomialNB(alpha=alpha)
    clf.fit(X_train, y_train)
    
    y_val_pred = clf.predict(X_val)
    f1 = f1_score(y_val, y_val_pred, average='weighted')  
    
    return f1
def Naive_Bayes_MN(X_train, X_val, y_train, y_val, n_trials=30):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_naive_MN(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False)
    
    print("Melhores hiperparâmetros encontrados:")
    print(study.best_params)
    print(f"Melhor F1 (validação): {study.best_value:.4f}")
    
    # Treinar o modelo final com o melhor alpha
    best_model = MultinomialNB(**study.best_params)
    best_model.fit(X_train, y_train)
    
    # Previsão no conjunto de teste
    y_test_pred = best_model.predict(X_train)
    
    return y_test_pred

#SVM ---------------------
def objective_svm(trial, X_train, X_val, y_train, y_val):

    #Definir hiperparâmetros a testar 
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3

    #Criar modelo SVM 
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)

    #treinar modelo com conjunto de treino
    model.fit(X_train, y_train)

    #prever no conjunto de validação
    y_pred = model.predict(X_val)

    #calcular f1
    f1 = f1_score(y_val, y_pred, average='macro')

    return f1

def svm(X_train, X_val, y_train, y_val, n_trials=30):
    # Chamar o Optuna para otimizar os parâmetros
    objeto_para_otimizar = optuna.create_study(direction='maximize')
    objeto_para_otimizar.optimize(lambda trial: objective_svm(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False )

    print("Melhores hiperparâmetros encontrados:")
    print(objeto_para_otimizar.best_params)

    # Treinar o melhor modelo
    best_model = SVC(**objeto_para_otimizar.best_params)
    best_model.fit(X_train, y_train)

    #  Avaliar no conjunto de teste
    y_pred_test = best_model.predict(X_train)

    return y_pred_test

#KNN---------------------------------------
def KNN(X_train, y_train, X_val, y_val):
    k_values = range(1, 200,10)  # Determinar  intervalo de valores a testar para k
    best_k = 1
    best_f1=0
    k_list = []
    f1_list = [] 

    for k in k_values:

        knn = KNeighborsClassifier(n_neighbors=k)  
        knn.fit(X_train, y_train)

        y_pred_KNN = knn.predict(X_val) 
        f1score = f1_score(y_val, y_pred_KNN, average= 'macro')

        k_list.append(k)
        f1_list.append(f1score)

        if f1score > best_f1: 
            best_f1 = f1score
            best_k = k

    knn = KNeighborsClassifier(n_neighbors=best_k)  
    knn.fit(X_train, y_train)

    y_pred_KNN= knn.predict(X_train) 
    print(f"Best k: {best_k}")

    return y_pred_KNN

#DECISION TREE--------------------
def objective_decision_tree(trial, X_train, X_val, y_train, y_val):
    # Hiperparâmetros a otimizar
    criterion = trial.suggest_categorical("criterion", ["entropy", "gini"])
    max_depth = trial.suggest_int("max_depth", 2, 50)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    ccp_alpha = trial.suggest_float("ccp_alpha", 0.0, 0.01)
    
    # Criar o modelo com os parâmetros sugeridos
    model = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        ccp_alpha=ccp_alpha,
        random_state=42
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    f1 = f1_score(y_val, y_pred, average="macro")

    return f1


def decision_tree(X_train, y_train, X_val, y_val, n_trials=30):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_decision_tree(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False)

    print("Melhores hiperparâmetros encontrados:")
    print(study.best_params)
    print("Melhor F1 obtido:", study.best_value)

    # Treinar o modelo final com os melhores parâmetros
    best_model = DecisionTreeClassifier(**study.best_params, random_state=42)
    best_model.fit(X_train, y_train)

    # Prever no conjunto de teste
    y_pred_test = best_model.predict(X_train)

    return y_pred_test

#RANDOM FLOREST---------------
def objective_random_forest(trial, X_train, X_val, y_train, y_val):
    # Hiperparâmetros a otimizar
    n_estimators = trial.suggest_int("n_estimators", 50, 300, step=50)
    max_depth = trial.suggest_int("max_depth", 5, 50)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

    # Criar o modelo
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        criterion=criterion,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    f1 = f1_score(y_val, y_pred, average="macro")

    return f1


def random_florest(X_train, y_train, X_val, y_val, n_trials=50):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_random_forest(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False)

    print("Melhores hiperparâmetros Random Forest:")
    print(study.best_params)
    print("Melhor F1 obtido:", study.best_value)

    best_model = RandomForestClassifier(**study.best_params, random_state=42, n_jobs=-1)
    best_model.fit(X_train, y_train)

    y_pred_test = best_model.predict(X_train)

    return y_pred_test

#NEURAL NETWORK------------------------

def objective_NN(X_train, y_train, X_val, y_val, trial):
    hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [50, 100, 200])
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    learning_rate_init = trial.suggest_float('learning_rate_init', 1e-4, 1e-2, log=True)

    clf = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        learning_rate_init=learning_rate_init,
        max_iter=300,
        random_state=42
    )

    clf.fit(X_train, y_train)
    preds = clf.predict(X_val)
    f1 = f1_score(y_val, preds, average='macro')
    return f1

def neural_network(X_train, y_train, X_val, y_val):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective_NN(X_train, y_train, X_val, y_val, trial), n_trials=30)
    print("Melhores parâmetros:", study.best_params)

    best_params = study.best_params
    best_model = MLPClassifier(
        hidden_layer_sizes=best_params['hidden_layer_sizes'],
        activation=best_params['activation'],
        learning_rate_init=best_params['learning_rate_init'],
        max_iter=300,
        random_state=42
    )

    best_model.fit(X_train, y_train)
    previsoes = best_model.predict(X_train)
    
    return previsoes

#REGRESSÃO LOGÍSTICA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import optuna

def objective_LR(X_train, y_train, X_val, y_val, trial):
    # Hiperparâmetros a otimizar
    C = trial.suggest_float('C', 1e-3, 1e3, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])  # compatíveis com L1 e L2
    max_iter = trial.suggest_int('max_iter', 100, 1000)

    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver=solver,
        max_iter=max_iter,
        random_state=42
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    f1 = f1_score(y_val, preds, average='macro')
    return f1


def logistic_regression(X_train, y_train, X_val, y_val):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective_LR(X_train, y_train, X_val, y_val, trial), n_trials=30)

    print("Melhores parâmetros encontrados:")
    print(study.best_params)

    best_params = study.best_params
    best_model = LogisticRegression(
        **best_params,
        random_state=42
    )

    best_model.fit(X_train, y_train)

    previsoes = best_model.predict(X_train)

    return previsoes

In [36]:
# Count Vectorizer
predicted_NB = Naive_Bayes_MN(X_train_count_vec,X_val_count_vec,y_train,y_val)
predicted_SVM= svm(X_train_count_vec, X_val_count_vec, y_train, y_val, n_trials=30) 
predicted_KNN= KNN(X_train_count_vec, y_train, X_val_count_vec, y_val) 
predicted_DT= decision_tree(X_train_count_vec, y_train, X_val_count_vec, y_val) 
predicted_RF= random_florest(X_train_count_vec, y_train, X_val_count_vec, y_val) 
predicted_NN= neural_network(X_train_count_vec, y_train, X_val_count_vec, y_val) 
predicted_LR= logistic_regression(X_train_count_vec, y_train, X_val_count_vec, y_val) 

Melhores hiperparâmetros encontrados:
{'alpha': 1.0339354147373747}
Melhor F1 (validação): 0.7621
Melhores hiperparâmetros encontrados:
{'C': 1.5498565056234925, 'kernel': 'linear', 'gamma': 'scale'}
Best k: 1
Melhores hiperparâmetros encontrados:
{'criterion': 'entropy', 'max_depth': 15, 'min_samples_split': 19, 'min_samples_leaf': 3, 'ccp_alpha': 0.0011693454017834804}
Melhor F1 obtido: 0.5479818820164344
Melhores hiperparâmetros Random Forest:
{'n_estimators': 150, 'max_depth': 34, 'max_features': None, 'criterion': 'entropy', 'min_samples_split': 6, 'min_samples_leaf': 2}
Melhor F1 obtido: 0.5514269905269964


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\neural_network\_multi

Melhores parâmetros: {'hidden_layer_sizes': 50, 'activation': 'relu', 'learning_rate_init': 0.00010394738464373503}


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
c:\Users\didia\anacond

Melhores parâmetros encontrados:
{'C': 2.9655647850643545, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 986}


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


In [2]:
'''predicts = pd.DataFrame({'predicted_train_NB': predicted_NB,
            'predicted_train_SVM': predicted_SVM,
            'predicted_train_KNN': predicted_KNN,
            'predicted_train_DT': predicted_DT,
            'predicted_train_RF': predicted_RF,
            'predicted_train_NN': predicted_NN,
            'predicted_train_LR': predicted_LR})
predicts["Real"] = y_train.values  
predicts["ID"] = predicts.index
predicts.to_csv('predicts_train.csv')
'''
predicts = pd.read_csv('predicts_train.csv')

In [55]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

def analisar_entropia(train, predicts):
    # Criar dataframe base
    merged = pd.DataFrame()
    merged["Real"] = train["classe"]
    merged["sentences"] = train["sentences"]

    # Adicionar previsões dos modelos
    for nome, preds in predicts.items():
        merged[nome] = preds

    merged["ID"] = merged.index

    # Função para calcular entropia de uma linha de previsões
    def calcular_entropia(preds_linha):
        _, contagens = np.unique(preds_linha, return_counts=True)
        probs = contagens / contagens.sum()
        return entropy(probs, base=2)

    # Calcular entropia para cada linha
    pred_cols = [c for c in merged.columns if c not in ["Real", "sentences", "ID"]]
    merged["Entropia"] = merged[pred_cols].apply(lambda r: calcular_entropia(r.values), axis=1)

    # Criar DataFrames por classe
    dfs_por_classe = {}
    for classe in merged["Real"].unique():
        df_c = merged[merged["Real"] == classe][["ID", "sentences", "Real", "Entropia"]]
        df_c = df_c.sort_values(by="Entropia", ascending=True).reset_index(drop=True)
        dfs_por_classe[classe] = df_c

    for classe, df_c in dfs_por_classe.items():
        print(f"\nClasse {classe}:")
        print(df_c.head())
        print(f"Valores distintos de entropia: {df_c['Entropia'].nunique()}")
        print(f"Entropia 0 nesta classe: {sum(df_c['Entropia'] == 0)}")

    return dfs_por_classe

# Exemplo de uso:
dfs_por_classe = analisar_entropia(train, predicts)
df_facto = dfs_por_classe.get(0)
df_vies = dfs_por_classe.get(1)
df_citacao = dfs_por_classe.get(-1)



Classe 0:
     ID                                          sentences  Real  Entropia
0     0  Neymar, que a partir de 20 de novembro vai lid...     0  0.000000
1  3463  Em 1997, viu sua relação com os britânicos ser...     0  0.543564
2  3417  Como outros países, o gigante asiático também ...     0  0.543564
3  3418  A Virginia Tech, com 26 mil alunos, fica a 390...     0  0.543564
4  3419  Mais tarde, em 1999, ele reprisaria o papel e ...     0  0.543564
Valores distintos de entropia: 7
Entropia 0 nesta classe: 1

Classe -1:
    ID                                          sentences  Real  Entropia
0  207                       O que nós queremos com isso?    -1  0.543564
1  217  A gente nunca imaginou que poderia estar do la...    -1  0.543564
2  219  "Webb, agora, está voando sozinho", informou a...    -1  0.543564
3  220  "Obstáculos:"No início da campanha o noticiári...    -1  0.543564
4  221  "É sobre isso, é sobre o idealismo da família,...    -1  0.543564
Valores distintos de en

### Abordagem com o k-means

In [130]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd

def kmeans(sentences, classes_reais, n_clusters=3):
    # Vetorização com CountVectorizer
    c_vect = CountVectorizer(
        token_pattern=r'\"|[A-Za-zÀ-ÿ]+|[^\w\s]', 
        ngram_range=(1, 3),    
        strip_accents='unicode',
        max_features=1000
    )
    
    vetores = c_vect.fit_transform(sentences).toarray() 

    # aplicar KMeans 
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(vetores)

    centroids = kmeans.cluster_centers_

    df = pd.DataFrame({
        "sentenca": sentences,
        "classe_real": classes_reais,
        "cluster": labels
    })

    print("\nClasses por cluster")
    dist = df.groupby(["cluster", "classe_real"]).size().unstack(fill_value=0)
    dist.head()

    # classe dominante por cluster
    classe_do_cluster = dist.idxmax(axis=1)
    print("Classe atribuída a cada cluster pela maioria:")
    print(classe_do_cluster, "\n")

    # frases mais representativas por cluster)
    print("\nFrases mais proximas de cada cluster por ordem")

    for c in range(n_clusters):
        ind = np.where(labels == c)[0]     
        #calcular distancias de cada frase ao centroide                    
        distancias = np.linalg.norm(vetores[ind] - centroids[c], axis=1)
        top3 = ind[np.argsort(distancias)[:3]]

        print(f"\nCluster {c} (classe provável: {classe_do_cluster[c]})\n")
        for i, idx in enumerate(top3):
            print(f"{i+1}. {sentences[idx]} Classe real: = {classes_reais[idx]}")

    return df, classe_do_cluster, centroids


In [ ]:
sentences = train["sentences"].astype(str).tolist()
classes = train["classe"].tolist()

df_clusters, classe_cluster, centroides = kmeans(sentences, classes, 3)


Classes por cluster
Classe atribuída a cada cluster pela maioria:
cluster
0    0
1    0
2    0
dtype: int64 


Frases mais proximas de cada cluster por ordem

Cluster 0 (classe provável: 0)

1. Em 2012, motivada pelo declínio de Berlusconi, criou a própria agremiação. Classe real: = 0
2. Em 2014, o livro virou peça de teatro, estrelada por Nicette Bruno. Classe real: = 0
3. A plataforma Conecte SUS, que fornece o certificado nacional de vacinação, também saiu do ar. Classe real: = 0

Cluster 1 (classe provável: 0)

1. O medalhista olímpico, que fez parte do quarteto das eliminatórias pela manhã, contou com a ajuda dos amigos de seleção Bruno Fratus, Matheus Santana, Marcelo Chierighini e João de Lucca para garantir sua 19ª medalha. Classe real: = 0
2. Por exemplo, os integralistas se opõem à teologia da libertação, uma espécie de campo progressista da religião, e ao aborto, inclusive em caso de estupro. Classe real: = 0
3. Autora de grandes sucessos, como Perdas e Ganhos, de 2003, que

Abordagem com a distância dos vetores do Count Vec

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd

def Count_vec_centroid(sentences, classes, top_n=3):
    
    # Vetorização
    vectorizer = CountVectorizer(token_pattern=r'\"|[A-Za-zÀ-ÿ]+|[^\w\s]', 
        ngram_range=(1, 3),    
        strip_accents='unicode',
        max_features=1000)
    X = vectorizer.fit_transform(sentences).toarray() 
    
    df = pd.DataFrame({
        "sentenca": sentences,
        "classe": classes
    })
    
    rows = []

    for classe in df["classe"].unique():
        idx_classe = df[df["classe"] == classe].index
        vetores_classe = X[idx_classe]
        
        # Calcula centróide da classe
        centroide = vetores_classe.mean(axis=0)
        
        # Distância Euclidiana ao centróide
        distancias = np.linalg.norm(vetores_classe - centroide, axis=1)
        
        # Top frases mais próximas do centróide
        top_idx = idx_classe[np.argsort(distancias)[:top_n]]

        for i, idx in enumerate(top_idx):
            rows.append({
                "Index": idx,
                "classe": int(classe),
                "rank": i+1,
                "sentenca": df.loc[idx, "sentenca"],
                "distancia_centroide": distancias[np.argsort(distancias)[:top_n]][i]
            })
    
    resultado_df = pd.DataFrame(rows)
    return resultado_df

sentences = train["sentences"].astype(str).tolist()
classes = train["classe"].tolist()
top_frases = Count_vec_centroid(sentences, classes, top_n=3)
print(top_frases)

   Index  classe  rank                                           sentenca  \
0   3309       0     1                  Artigo 14, presunção de inocência   
1   1904       0     2            Novamente, Kaká chutou de fora da área.   
2   1798       0     3       Ameaças de Putin enfraquecem o tabu nuclear.   
3   1932      -1     1                            É padrão Zâmbia, Gabão.   
4    324      -1     2   Mostrei toda a minha indignação ali", completou.   
5     61      -1     3                Não sejam tocados, abracem a causa.   
6   1260       1     1  Aos 19, o atacante do CSKA perdeu grande chanc...   
7   1113       1     2                    Aos 32min, Kaká tentou de novo.   
8   2290       1     3                       A posição de Taiwan é firme:   

   distancia_centroide  
0             2.026610  
1             2.106179  
2             2.184103  
3             1.877806  
4             2.006368  
5             2.010203  
6             2.249491  
7             2.391749  
8   

abordagem da entropia com mais vetorizadores e modelos de ML

In [ ]:
DT = pd.read_csv('predict_DT.csv')
KNN = pd.read_csv('predict_KNN.csv')
NB = pd.read_csv('predict_NB.csv')
NN = pd.read_csv('predict_NN.csv')
SVM = pd.read_csv('predict_SVM.csv')

In [61]:
modelos = {
    "NB": NB,
    "SVM": SVM,
    "KNN": KNN,
    "DT": DT,
    "NN": NN
}

In [62]:
from scipy.stats import entropy
import numpy as np

def calcular_entropia(preds_linha):
    _, contagens = np.unique(preds_linha, return_counts=True)
    probs = contagens / contagens.sum()
    return entropy(probs, base=2)

In [ ]:
entropias = []
acertos_frac = []

for idx in top_frases["Index"]:
    preds_linha = []
    acerto = 0
    total = 0
    
    classe_real = top_frases.loc[top_frases["Index"] == idx, "classe"].values[0]
    
    for nome, df in modelos.items():
        cols = [c for c in df.columns if c != "Real"]
        valores = df.loc[idx, cols].values
        preds_linha.extend(valores)
        
        acerto += sum(valores == classe_real)
        total += len(cols)
    
    entropias.append(calcular_entropia(preds_linha))
    acertos_frac.append(acerto / total)  

top_frases["Entropia"] = entropias
top_frases["Acertos_frac"] = acertos_frac

print(top_frases)


   Index  classe  rank                                           sentenca  \
0   3309       0     1                  Artigo 14, presunção de inocência   
1   1904       0     2            Novamente, Kaká chutou de fora da área.   
2   1798       0     3       Ameaças de Putin enfraquecem o tabu nuclear.   
3   1932      -1     1                            É padrão Zâmbia, Gabão.   
4    324      -1     2   Mostrei toda a minha indignação ali", completou.   
5     61      -1     3                Não sejam tocados, abracem a causa.   
6   1260       1     1  Aos 19, o atacante do CSKA perdeu grande chanc...   
7   1113       1     2                    Aos 32min, Kaká tentou de novo.   
8   2290       1     3                       A posição de Taiwan é firme:   

   distancia_centroide  Entropia  Acertos  Acertos_frac  
0             2.026610  0.210842       29      0.966667  
1             2.106179  0.699843       26      0.866667  
2             2.184103  1.052982       22      0.733333

In [59]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

train = train.reset_index(drop=True)

#juntar as previsões horizontalmente
merged = pd.DataFrame({
    "Real": NB["Real"],      # coluna y reais             
    "NB":  NB.iloc[:,1],
    "SVM": SVM.iloc[:,1],
    "KNN": KNN.iloc[:,1],
    "DT": DT.iloc[:,1],    #colunas dos previstos
    "NN": NN.iloc[:,1],
})

merged["ID"] = merged.index #guarda o indice p depois ir buscar a frase

#entropia = - somatório (prob_classe . log (prob_classe))
#prob_classe ( n vezes classificada / todas as classificações)
def calcular_entropia(preds):
    _, contagens = np.unique(preds, return_counts=True)
    probs = contagens / contagens.sum()
    return entropy(probs, base=2)

# Para cada modelo, seleciona todas as colunas de previsão exceto "Real"
dfs = []
for df, nome in zip([NB, SVM, KNN, DT, NN],
                    ["NB","SVM","KNN","DT","NN"]):
    cols = [c for c in df.columns if c != "Real"]
    # renomeia as colunas para não haver conflito
    df_renamed = df[cols].add_prefix(f"{nome}_")
    dfs.append(df_renamed)

# concatena horizontalmente
merged = pd.concat(dfs, axis=1)
merged["Real"] = NB["Real"].values   # y real
merged["sentences"] = train["sentences"]

# Calcular entropia para cada linha
pred_cols = [c for c in merged.columns if c not in ["Real", "sentences"]]
merged["Entropia"] = merged[pred_cols].apply(lambda r: calcular_entropia(r.values), axis=1)

# Criar DataFrames separados por classe
dfs_por_classe = {}
for classe in merged["Real"].unique():
    df_c = merged[merged["Real"] == classe][["sentences","Real","Entropia"]]
    df_c = df_c.sort_values(by="Entropia", ascending=True).reset_index(drop=True)
    dfs_por_classe[classe] = df_c

# Extrair dataframes
df_facto   = dfs_por_classe.get(0)
df_vies    = dfs_por_classe.get(1)
df_citacao = dfs_por_classe.get(-1)

print("facto:")
print(df_facto.head())
print(f"Entropia 0 nesta classe: {sum(df_facto['Entropia'] == 0)}")

print("\nvies")
print(df_vies.head())
print(f"Entropia 0 nesta classe: {sum(df_vies['Entropia'] == 0)}")

print("\ncitacao")
print(df_citacao.head())
print(f"Entropia 0 nesta classe: {sum(df_citacao['Entropia'] == 0)}")

facto:
                                           sentences  Real  Entropia
0  Em 2012, motivada pelo declínio de Berlusconi,...     0       0.0
1  Formulador do Real, André Lara Resende declara...     0       0.0
2  O presidente de honra da Beija-Flor de Nilópol...     0       0.0
3  Ele não recebeu o convite para participar da r...     0       0.0
4  Com o resultado, a inflação acumulada no ano d...     0       0.0
Entropia 0 nesta classe: 102

vies
                                           sentences  Real  Entropia
0  O gol deu a tranqüilidade necessária ao time b...     1  0.783777
1  Na semana passada, Lira manobrou para facilita...     1  0.783777
2  Brasil deslancha no fim e goleia Equador em su...     1  0.783777
3  Com sua escrita potente, elogiada pela imortal...     1  0.836641
4  Livre de marcação, o atacante --após seis part...     1  0.836641
Entropia 0 nesta classe: 0

citacao
                                sentences  Real  Entropia
0  ?Isso não é um blefe?, destacou P

In [ ]:
svm_over = pd.read_csv('predicts_svm_over.csv')
DT_over = pd.read_csv('predict_DT_over.csv')
KNN_over = pd.read_csv('predict_KNN_over.csv')
NB_over= pd.read_csv('predict_NB_over.csv')

In [58]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

train = train.reset_index(drop=True)

#juntar as previsões horizontalmente
merged = pd.DataFrame({
    "Real": NB_over["Real"],      # coluna y reais             
    "NB":  NB_over.iloc[:,1],
    "SVM": svm_over.iloc[:,1],
    "KNN": KNN_over.iloc[:,1],
    "DT": DT_over.iloc[:,1],    #colunas dos previstos
})

merged["ID"] = merged.index #guarda o indice p depois ir buscar a frase

#entropia = - somatório (prob_classe . log (prob_classe))
#prob_classe ( n vezes classificada / todas as classificações)
def calcular_entropia(preds):
    _, contagens = np.unique(preds, return_counts=True)
    probs = contagens / contagens.sum()
    return entropy(probs, base=2)

# Para cada modelo, seleciona todas as colunas de previsão exceto "Real"
dfs = []
for df, nome in zip([NB_over, svm_over, KNN_over, DT_over],
                    ["NB_over","svm_over","KNN_over","DT_over"]):
    cols = [c for c in df.columns if c != "Real"]
    # renomeia as colunas para não haver conflito
    df_renamed = df[cols].add_prefix(f"{nome}_")
    dfs.append(df_renamed)

# concatena horizontalmente
merged = pd.concat(dfs, axis=1)
merged["Real"] = NB_over["Real"].values   # y real
merged["sentences"] = train["sentences"]

# Calcular entropia para cada linha
pred_cols = [c for c in merged.columns if c not in ["Real", "sentences"]]
merged["Entropia"] = merged[pred_cols].apply(lambda r: calcular_entropia(r.values), axis=1)

# Criar DataFrames separados por classe
dfs_por_classe = {}
for classe in merged["Real"].unique():
    df_c = merged[merged["Real"] == classe][["sentences","Real","Entropia"]]
    df_c = df_c.sort_values(by="Entropia", ascending=True).reset_index(drop=True)
    dfs_por_classe[classe] = df_c

# Extrair dataframes
df_facto   = dfs_por_classe.get(0)
df_vies    = dfs_por_classe.get(1)
df_citacao = dfs_por_classe.get(-1)

print("facto:")
print(df_facto.head())
print(f"Valores distintos de entropia: {df_facto['Entropia'].nunique()}")
print(f"Entropia 0 nesta classe: {sum(df_facto['Entropia'] == 0)}")

print("\nvies")
print(df_vies.head())
print(f"Valores distintos de entropia: {df_vies['Entropia'].nunique()}")
print(f"Entropia 0 nesta classe: {sum(df_vies['Entropia'] == 0)}")

print("\ncitacao")
print(df_citacao.head())
print(f"Valores distintos de entropia: {df_citacao['Entropia'].nunique()}")
print(f"Entropia 0 nesta classe: {sum(df_citacao['Entropia'] == 0)}")

facto:
                                           sentences  Real  Entropia
0  Ambos os trens se dirigiam ao sul e levavam mo...     0  0.391244
1  O Deic diz que prendeu o garçom e o grafiteiro...     0  0.391244
2  As ações são julgadas no plenário virtual do S...     0  0.391244
3  Ramos Pereira disse que o encontro "terá desdo...     0  0.391244
4  A Reuters não conseguiu aferir a veracidade da...     0  0.391244
Valores distintos de entropia: 107
Entropia 0 nesta classe: 0

vies
                                           sentences  Real  Entropia
0  Aos 14, porém, Juan lançou Vagner Love, que ap...     1  0.391244
1         Em tom de desafio, Almeida Lima respondeu:     1  0.391244
2  O telescópio James Webb, da agência aeroespaci...     1  0.391244
3  O Senado tem freado a análise de propostas vot...     1  0.391244
4                  A partida começou muito amarrada.     1  0.391244
Valores distintos de entropia: 84
Entropia 0 nesta classe: 0

citacao
                           